In [4]:
import pandas as pd
import numpy as np

In [22]:
## Select consensus level for finphrase

# filename = 'Sentences_66Agree.txt'
# filename = 'Sentences_50Agree.txt'
filename = 'Sentences_75Agree.txt'
# filename = 'Sentences_AllAgree.txt'

In [14]:
def load_finphrase(filename):
    ''' Clean FinancialPhrasebank data
        Input:
            - filename
        Output:
            - a dataframe for the loaded financial phase bank data
    '''
    df = pd.read_csv('../../finphrase_dir/' + filename,
                     sep='\@',
                     engine='python',
                     header=None,
                     names=['sentence','label'])
    df.drop_duplicates(inplace=True)
    print('Total number of record in the file: ', df.shape[0])
    df.reset_index(inplace=True, drop=True)
    return df

In [21]:
train_df = load_finphrase(filename)
train_df.sample(n=20)

Total number of record in the file:  3448


,sentence,label
1815,"`` After the transaction , Tikkurila has no powder coatings-related operations , '' the company said .",neutral
732,"cents The profile contains business operations , the company history , major products and services , prospects , key competitors , structure and key employees , locations and subsidiaries .",neutral
752,"Only this time , Nokia , India 's largest MNC , has sought out a topic that could spark off a million approaches ` Creativity in Emerging Markets .",neutral
1664,Ramirent Finland is the domestic unit of machinery rental company Ramirent Oyj HEL : RMR1V .,neutral
711,Uponor improved its performance in spite of the decrease in residential construction in the US .,positive
555,Major Order in India Comptel Corporation has received a significant long-term order for mediation and provisioning solutions being used by a leading operator in India .,positive
2069,"Nordstjernan will make the offer in approximately one month , in September 2007 .",neutral
2098,"Proline Plus is available in both adjustable single and multichannel models and fixed volume single-channel models , in volume ranges from 0.1 micro litres to 10 ml .",neutral
1995,"In the fourth quarter of 2006 , OKO Banks expects the operating environment for Banking and Investment Services to remain similar to that in January-September 2006 .",neutral
140,"Both operating profit and turnover for the six-month period increased , respectively , from EUR17 .6 m and EUR1149 .9 m , as compared to the corresponding period in 2007 .",positive


In [3]:
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, BertModel
from transformers import AdamW as AdamW_HF, get_linear_schedule_with_warmup

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset

In [ ]:
class BERT_model(nn.Module):
    def __init__(self, hidden_size, dense_size, output_size, dropout=0.1):
        """
        Initialize the model
        """
        super().__init__()
        self.output_size = output_size
        self.dropout = dropout
        
        self.bert = BertModel.from_pretrained('bert-base-uncased',  
                                        output_hidden_states=True,
                                        output_attentions=True)
        for param in self.bert.parameters():
            param.requires_grad = True
        self.weights = nn.Parameter(torch.rand(13, 1))
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size, dense_size)
        self.fc2 = nn.Linear(dense_size, output_size)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_ids):
        """
        Perform a forward pass of the model on nn_input
        """
        
        all_hidden_states, all_attentions = self.bert(input_ids)[-2:]
        batch_size = input_ids.shape[0]
        ht_cls = torch.cat(all_hidden_states)[:, :1, :].view(13, batch_size, 1, 768)
        atten = torch.sum(ht_cls * self.weights.view(13, 1, 1, 1), dim=[1, 3])
        atten = F.softmax(atten.view(-1), dim=0)
        feature = torch.sum(ht_cls * atten.view(13, 1, 1, 1), dim=[0, 2])        
        # Dense layer
        dense_out = self.fc1(self.dropout(feature))
        out = self.fc2(dense_out)

        return out